# RSP Visualization (Simulated Data)

This script creates visualizations of RSP (Radar Scanning Plot) using simulated data. We first generate background points uniformly distributed within a unit circle, then create foreground subsets for multiple genes with optional angular bias patterns.

This allows us to test the RSP algorithm with controlled spatial distributions and validate that the package correctly detects directional clustering patterns.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np

In [2]:
import spatialrsp as rsp

In [3]:
import logging

logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s %(name)s %(levelname)-8s %(message)s"
)

In [4]:
plt.rcParams["figure.dpi"] = 400
plt.rcParams["font.size"] = 14
plt.rcParams["axes.grid"] = True

distinct_colors = [
    "#d62728",
    "#2ca02c",
    "#ff7f0e",
    "#1f77b4",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#bcbd22",
    "#17becf",
]

In [5]:
class DummyAnnData:
    def __init__(self, angles, radii):
        self.obsm = {
            "X_polar": np.column_stack((radii, angles)),
            "X_umap": np.column_stack((radii * np.cos(angles), radii * np.sin(angles))),
        }

A neat script that I developed to selectively generate points in a unit circle with optional angular bias patterns.

In [6]:
def generate_points(n_cells, mode="random", bias_angles=None, bias_weights=None):
    radii = np.sqrt(np.random.uniform(0, 1, size=n_cells))
    if mode == "random":
        angles = np.random.uniform(0, 2 * np.pi, size=n_cells)
    elif mode == "biased":
        if bias_angles is None:
            raise ValueError("bias_angles must be provided in 'biased' mode")
        bias_angles = np.atleast_1d(bias_angles)
        if bias_weights is None:
            bias_weights = np.ones(len(bias_angles)) / len(bias_angles)
        else:
            bias_weights = np.array(bias_weights, dtype=float)
            bias_weights = bias_weights / bias_weights.sum()
        idx = np.random.choice(len(bias_angles), size=n_cells, p=bias_weights)
        angles = bias_angles[idx] + np.random.normal(scale=0.1, size=n_cells)
        angles = np.mod(angles, 2 * np.pi)
    else:
        raise ValueError(f"Unknown mode: {mode}")
    return angles, radii

In [7]:
n_cells = 100000
angles = np.random.uniform(0, 2 * np.pi, size=n_cells)
radii = np.sqrt(np.random.uniform(0, 1, size=n_cells))

filtered_cells = DummyAnnData(angles, radii)
bg_angles = angles

In [8]:
genes = ["geneA", "geneB"]
coverage = {"geneA": 0.10, "geneB": 0.10}
thresholds = coverage.copy()

bias_params = {
    "geneA": None,
    "geneB": {"mean": np.pi / 2, "kappa": 5},
}

In [9]:
fg_masks = {}
fg_angles_list = []
for g in genes:
    cov = coverage[g]
    if bias_params[g] is None:
        mask = np.random.rand(n_cells) < cov
    else:
        bp = bias_params[g]
        w = np.exp(bp["kappa"] * np.cos(angles - bp["mean"]))
        w = w / w.sum()
        n_fg = int(cov * n_cells)
        idx = np.random.choice(n_cells, size=n_fg, replace=False, p=w)
        mask = np.zeros(n_cells, dtype=bool)
        mask[idx] = True
    fg_masks[g] = mask
    fg_angles_list.append(angles[mask])

In [10]:
scanning_window = np.pi
scanning_range = np.linspace(0, 2 * np.pi, 360)
resolution = 100
mode = "relative"
expected_model = "local"
normalize = True

In [ ]:
if mode == "absolute":
    fg_curves, exp_curves, bg_curve = rsp.compute_rsp(
        theta_fgs=fg_angles_list,
        theta_bg=bg_angles,
        scanning_window=scanning_window,
        scanning_range=scanning_range,
        resolution=resolution,
        mode=mode,
        expected_model=expected_model,
        normalize=normalize,
    )
elif mode == "relative":
    fg_curves, bg_curve = rsp.compute_rsp(
        theta_fgs=fg_angles_list,
        theta_bg=bg_angles,
        scanning_window=scanning_window,
        scanning_range=scanning_range,
        resolution=resolution,
        mode=mode,
        expected_model=expected_model,
        normalize=normalize,
    )
else:
    raise ValueError(f"Unknown mode: {mode}. Use 'absolute' or 'relative'.")

In [ ]:
def plot_rsp_visualization(
    filtered_cells,
    genes,
    fg_masks,
    thresholds,
    fg_curves,
    bg_curve,
    scanning_range,
    mode="absolute",
    exp_curves=None,
    polar_umap=False,
):
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(1, 2, figure=fig, width_ratios=[5, 4])

    if polar_umap:
        ax1 = fig.add_subplot(gs[0], projection="polar")
        polar_coords = filtered_cells.obsm["X_polar"]
        umap_coords = np.column_stack((polar_coords[:, 1], polar_coords[:, 0]))
        coord_labels = ["Angle", "Radius"]
    else:
        ax1 = fig.add_subplot(gs[0])
        umap_coords = filtered_cells.obsm["X_umap"]
        coord_labels = ["UMAP1", "UMAP2"]

    colors = distinct_colors[: len(genes)]
    ax1.scatter(umap_coords[:, 0], umap_coords[:, 1], c="gray", s=1, label="Background")
    for i, gene in enumerate(genes):
        fg_mask = fg_masks[gene]
        threshold_val = thresholds[gene]

        ax1.scatter(
            umap_coords[fg_mask, 0],
            umap_coords[fg_mask, 1],
            c=colors[i],
            s=1,
            label=f"{gene} (thr: {threshold_val:.2f})",
        )
    ax1.legend(loc="upper right", fontsize=10)
    if not polar_umap:
        ax1.set_xlabel(coord_labels[0], fontsize=14)
        ax1.set_ylabel(coord_labels[1], fontsize=14)
        ax1.set_aspect("equal")

    ax1.tick_params(labelsize=12)

    ax2 = fig.add_subplot(gs[1], projection="polar")
    theta = np.asarray(scanning_range)
    n = len(fg_curves[0])

    if theta.size == n + 1 and np.isclose((theta[-1] - theta[0]) % (2 * np.pi), 0.0):
        theta = theta[:-1]
    elif theta.size == 2:
        start, end = theta
        theta = np.linspace(start, end, n, endpoint=False)
    elif theta.size != n:
        raise ValueError(f"scanning_range length {theta.size} but fg_curve length {n}")

    theta_closed = np.concatenate([theta, [theta[0]]])
    bg_closed = np.concatenate([bg_curve, [bg_curve[0]]])

    ax2.plot(
        theta_closed, bg_closed, ":", c="darkgray", label="Background", linewidth=1
    )

    for i, gene in enumerate(genes):
        fg_curve = fg_curves[i]
        fg_closed = np.concatenate([fg_curve, [fg_curve[0]]])

        ax2.plot(
            theta_closed,
            fg_closed,
            c=colors[i],
            alpha=0.8,
            label=f"{gene} (observed)",
            linewidth=2,
            linestyle="-",
        )

        if mode == "absolute" and exp_curves is not None:
            exp_curve = exp_curves[i]
            exp_closed = np.concatenate([exp_curve, [exp_curve[0]]])

            ax2.plot(
                theta_closed,
                exp_closed,
                c=colors[i],
                alpha=0.6,
                label=f"{gene} (expected)",
                linewidth=1.5,
                linestyle="--",
            )
    ax2.legend(loc="lower right", bbox_to_anchor=(1.1, -0.22), fontsize=10)
    ax2.tick_params(labelsize=12)

    plt.tight_layout()
    return fig, (ax1, ax2)


print("=== Plotting with Cartesian UMAP ===")
fig_cart, axes_cart = plot_rsp_visualization(
    filtered_cells=filtered_cells,
    genes=genes,
    fg_masks=fg_masks,
    thresholds=thresholds,
    fg_curves=fg_curves,
    bg_curve=bg_curve,
    scanning_range=scanning_range,
    mode=mode,
    exp_curves=exp_curves if mode == "absolute" else None,
    polar_umap=False,
)
plt.show()

print("=== Plotting with Polar UMAP ===")
fig_polar, axes_polar = plot_rsp_visualization(
    filtered_cells=filtered_cells,
    genes=genes,
    fg_masks=fg_masks,
    thresholds=thresholds,
    fg_curves=fg_curves,
    bg_curve=bg_curve,
    scanning_range=scanning_range,
    mode=mode,
    exp_curves=exp_curves if mode == "absolute" else None,
    polar_umap=True,
)
plt.show()

## Interpretation

The visualization above shows:

1. **Left panel (UMAP)**: The spatial distribution of cells expressing each gene above the percentile threshold
   - Gray dots: Background cells (low expression)
   - Colored dots: Cells with high expression for each gene
   - Legend shows the actual threshold value for each gene
2. **Right panel (RSP - Absolute Mode)**: The radar scanning plot showing angular enrichment patterns
   - **Solid lines**: Observed spatial enrichment for each gene
   - **Dashed lines**: Expected enrichment under the local background model
   - **Dotted gray line**: Background reference curve
   - Values represent absolute spatial clustering strength

### RSP Curve Interpretation

- **Observed > Expected**: Gene shows stronger spatial clustering than expected
- **Observed < Expected**: Gene shows weaker spatial clustering than expected
- **Observed ≈ Expected**: Gene follows expected spatial distribution patterns
- **Peak directions**: Indicate preferred spatial orientations for gene expression
